In [28]:
## https://docs.python.org/3/library/threading.html

In [5]:
import threading
from threading import Thread

import logging

FORMAT = '%(asctime)-15s %(threadName)s: %(message)s'

logging.basicConfig(level=logging.INFO, format=FORMAT)
logger = logging.getLogger(__name__)

In [6]:
def where_am_i():
    print("Now I am running %s the main thread" % ('IN ' if threading.current_thread() == threading.main_thread() else 'NOT IN '))

def heavy_work():
    int_log = logging.getLogger(__name__)
    where_am_i()
    print("Performing some actions ... ")
    int_log.info("Exit the thead")
    
    
where_am_i()
    
t = Thread(name='MyThread', target=heavy_work)
t.start()




2018-09-04 15:12:23,511 MyThread: Exit the thead


Now I am running IN  the main thread
Now I am running NOT IN  the main thread
Performing some actions ... 


In [7]:

class MyThread(Thread):
    def __init__(self):
        super().__init__()
        
    def run(self):
        print("This is actually what this thread is doing...")
        
        
mt = MyThread()
mt.start()

This is actually what this thread is doing...


In [39]:
import threading
import time
from threading import Lock

## A bus seats booking
## Actions: Book, pay, confirm


def remaining_seats(initial):
    if initial <= 0:
        return None
    
    number = 1
    while number <= initial:
        yield ("#{0:0" + str(len(str(initial))) + "d}").format(number)
        number += 1
        
    return None

class Cashier(Thread):    
    def __init__(self, lock, seats, cashier_name):
        super().__init__(name=cashier_name)
        self.lock = lock
        self.seats = seats
        
        
    def run(self):

        while True:
            self.lock.acquire()
            try:
                ticket = next(bus_seats)
                logger.info("Solding out a ticket %s ..." % ticket)
                time.sleep(0.005)
                logger.info("Ticket %s is sold now" % ticket)
            except StopIteration as sierr:
                logger.info("No more tickets left. The cashier is closing now ...")
                return;
            finally:
                self.lock.release()

            time.sleep(0.001)
        
        
bus_seats = remaining_seats(20)
booking_lock = Lock()

c1 = Cashier(booking_lock, bus_seats, "Cashier #1")
c2 = Cashier(booking_lock, bus_seats, "Cashier #2")
c3 = Cashier(booking_lock, bus_seats, "Cashier #3")

c1.start()
c2.start()
c3.start()

2018-09-04 15:34:33,058 Cashier #1: Solding out a ticket #01 ...
2018-09-04 15:34:33,068 Cashier #1: Ticket #01 is sold now
2018-09-04 15:34:33,068 Cashier #2: Solding out a ticket #02 ...
2018-09-04 15:34:33,077 Cashier #2: Ticket #02 is sold now
2018-09-04 15:34:33,078 Cashier #3: Solding out a ticket #03 ...
2018-09-04 15:34:33,087 Cashier #3: Ticket #03 is sold now
2018-09-04 15:34:33,088 Cashier #1: Solding out a ticket #04 ...
2018-09-04 15:34:33,094 Cashier #1: Ticket #04 is sold now
2018-09-04 15:34:33,094 Cashier #2: Solding out a ticket #05 ...
2018-09-04 15:34:33,101 Cashier #2: Ticket #05 is sold now
2018-09-04 15:34:33,102 Cashier #3: Solding out a ticket #06 ...
2018-09-04 15:34:33,108 Cashier #3: Ticket #06 is sold now
2018-09-04 15:34:33,109 Cashier #1: Solding out a ticket #07 ...
2018-09-04 15:34:33,116 Cashier #1: Ticket #07 is sold now
2018-09-04 15:34:33,117 Cashier #2: Solding out a ticket #08 ...
2018-09-04 15:34:33,124 Cashier #2: Ticket #08 is sold now
2018-09-

In [44]:

class SimpleThread():
    def work(self, a):
        print(a)
        
st = SimpleThread()
        
Thread(target=st.work, args=('Hello from thread',)).start()



Hello from thread


In [48]:

from threading import Condition

class Channel():

    def __init__(self):
        self.__data = None
        
    @property
    def ready(self):
        return self.__data is not None

    @property
    def data(self):
        d = self.__data
        self.__data = None
        return d
    
    @data.setter
    def data(self, data):
        self.__data = data
    

class Producer(Thread):
    def __init__(self, condition, channel):
        super().__init__()
        self.name="Producer Thread"
        self.__condition = condition
        self.__channel = channel

    
    def run(self):
        time.sleep(1)
        with self.__condition:
            self.__channel.data = "DATA 34-0563-0"
    

class Consumer(Thread):
    def __init__(self, condition, channel):
        super().__init__()
        self.name="Consumer Thread"
        self.__condition= condition
        self.__channel = channel

    
    def run(self):
        with self.__condition:
            cv.wait_for(self.__channel.ready)
            data = self.__channel.data
            print(data)


cv = Condition()
ch = Channel()

p = Producer(cv, ch)
c = Consumer(cv, ch)

c.start()
p.start()



Exception in thread Consumer Thread:
Traceback (most recent call last):
  File "c:\opt\python\3.7\lib\threading.py", line 917, in _bootstrap_inner
    self.run()
  File "<ipython-input-48-bc23b9f9a820>", line 48, in run
    cv.wait_for(self.__channel.ready)
  File "c:\opt\python\3.7\lib\threading.py", line 322, in wait_for
    result = predicate()
TypeError: 'bool' object is not callable

